In [1]:
from notebooks.ford.vins import search_vins

vins = search_vins(searches=None, include_openpilot=False)

Loaded 1264 VINs (filter_comment=None, include_openpilot=False, skipped=81)
Loaded AsBuilt data for 1264 VINs


In [2]:
import pandas as pd

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.ecu import FordEcu
from panda.python.uds import DATA_IDENTIFIER_TYPE

ECUS = [
  # FordEcu.AntiLockBrakeSystem,
  # FordEcu.CruiseControlModule,
  FordEcu.ImageProcessingModuleA,
  # FordEcu.PowerSteeringControlModule,
]

DID_PART_NUMBER = 0xF111

def print_identifiers(ecu):
  rows = set()
  for vin in vins:
    data = AsBuiltData.from_vin(vin)
    part = data.get_identifier(ecu, DID_PART_NUMBER)
    fw = data.get_identifier(ecu, DATA_IDENTIFIER_TYPE.VEHICLE_MANUFACTURER_ECU_SOFTWARE_NUMBER)
    rows.add((part, fw))
  return pd.DataFrame(rows, columns=['part', 'fw'])

for ecu in ECUS:
  print(f'{ecu.name} ({hex(ecu)})')
  print(print_identifiers(ecu))
  print()

ImageProcessingModuleA (0x706)
Unknown ECU address: 1808 (0x710)
0x710 {61706: 'NB3C-14E098-BC', 61712: 'DSNB3V-14E090-AB', 61713: 'NB3V-14E099-AA', 61715: 'NB3V-14E090-AB', 61832: 'NB3V-14E093-AB', 61836: '0000000002417620', 61920: 'PXNB3V-14E090-AA'}
Unknown ECU address: 2002 (0x7d2)
0x7d2 {61712: 'DSNL38-10D678-AG', 61713: 'NL38-10D686-AC', 61715: 'NL38-10D678-YC', 61732: 'NL38-10D682-AL', 61832: 'NL38-10D681-AL', 61836: 'DVMUA22323J01321'}
Unknown ECU address: 1808 (0x710)
0x710 {61706: 'NB3C-14E098-BB', 61712: 'DSNB3V-14E090-AB', 61713: 'NB3V-14E099-AA', 61715: 'NB3V-14E090-AB', 61832: 'NB3V-14E093-AB', 61836: '0000000002312882', 61920: 'PXNB3V-14E090-AA'}
Unknown ECU address: 2002 (0x7d2)
0x7d2 {61712: 'DSNL38-10D678-AG', 61713: 'NL38-10D686-AD', 61715: 'NL38-10D678-YD', 61732: 'NL38-10D682-AL', 61832: 'NL38-10D681-AL', 61836: 'DVMUA23184J00719'}
Unknown ECU address: 2002 (0x7d2)
0x7d2 {61712: 'DSNL38-10D678-AG', 61713: 'NL38-10D686-AD', 61715: 'NL38-10D678-YD', 61732: 'NL38-10D6

In [3]:
def parse_part_number(part_number):
  prefix, core, suffix = part_number.split('-')
  return {
    'prefix': prefix,
    'core': core,
    'suffix': suffix,
  }

def print_core(ecu):
  rows = set()
  for vin in vins:
    data = AsBuiltData.from_vin(vin)
    part = data.get_identifier(ecu, DID_PART_NUMBER)
    if part:
      parts = parse_part_number(part)
      part = parts['core']
    rows.add(part)
  return pd.DataFrame(rows)

for ecu in ECUS:
  print(f'{ecu.name} ({hex(ecu)})')
  print(print_core(ecu))
  print()

ImageProcessingModuleA (0x706)
        0
0  14H107
1    None
2  14G025
3  14F403



In [4]:
from collections import defaultdict


def print_configuration_size(ecu):
  rows = []
  for vin in vins:
    data = AsBuiltData.from_vin(vin)
    part = data.get_identifier(ecu, DID_PART_NUMBER)
    if not part:
      continue
    config = data.get_configuration(ecu)
    if not config:
      rows.append({'part': part})
      continue
    blocks = defaultdict(set)
    for field in config.keys():
      block_no, field_no = field.split("-")
      blocks[block_no].add(field_no)
    blocks = dict({block_no: len(fields) for block_no, fields in blocks.items()})
    blocks['part'] = part
    rows.append(blocks)

  df = pd.DataFrame.from_records(rows)
  df.drop_duplicates(inplace=True, ignore_index=True)
  for col in df.columns:
    if col == 'part':
      continue
    df[col] = df[col].astype('Int64')

  df = df.join(df['part'].apply(parse_part_number).apply(pd.Series))
  df.drop(columns=['part'], inplace=True)

  df = df.reindex(sorted(df.columns), axis=1)
  df.fillna(0, inplace=True)
  return df

for ecu in ECUS:
  print(f'{ecu.name} ({hex(ecu)})')
  df = print_configuration_size(ecu)
  # print(df)

  constants = dict()
  for col in df.columns:
    if col in ('prefix', 'core', 'suffix'):
      continue
    if df[col].nunique() == 1:
      constants[col] = df[col].iloc[0]
  print(f'constants: {constants}')
  df.drop(columns=constants.keys(), inplace=True)

  print()

  # try grouping by everything EXCEPT part number
  for part, group in df.groupby([col for col in df.columns if col not in ('prefix', 'core', 'suffix')]):
    print(part)
    print(group)
    print()

  print()
  print()
  print()

ImageProcessingModuleA (0x706)
constants: {}

(2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
    01  02  03  04  05  06  07  08  09  10  ...  32  33  34  35  36  37  38  \
27   2   2   1   1   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   

      core  prefix  suffix  
27  14G025    HS7T      CC  

[1 rows x 41 columns]

(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 6, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0)
    01  02  03  04  05  06  07  08  09  10  ...  32  33  34  35  36  37  38  \
47   2   2   2   2   2   2   2   2   2   2  ...   0   2   2   0   0   0   0   
56   2   2   2   2   2   2   2   2   2   2  ...   0   2   2   0   0   0   0   

      core  prefix  suffix  
47  14H107    RL3T      CA  
56  14H107    PR3T     BBE  

[2 rows x 41 columns]

(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 6, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0)
    01  02  03  04 